Downloading Test Data

This dataset gives us ~42K text chunks to embed, each roughly a paragraph or two.

In [8]:
from datasets import load_dataset
# https://huggingface.co/datasets/jamescalam/ai-arxiv-chunked/viewer/default/train?p=0
# data = load_dataset(
#     "jamescalam/ai-arxiv-chunked",
#     split= "train")


data = load_dataset("json", data_files="train.jsonl", split="train")    



Prerequisites

In [9]:
import os
import cohere
import openai

# initialize cohere

co = cohere.Client()

# openai doesn't need to be initialized, but need to set api key


## Creating Embeddings

create an embed function for each model -  return a list of vector embeddings from string list


### hugginface; load model & embedding function

In [10]:
from embedding_models.hugginface import load_LLM,embed


# model_id = "intfloat/e5-base-v2"
model_id = "C:/models/e5-base-v2"

model, tokenizer, device = load_LLM(model_id)

device: cpu


### cohere embedding function

In [11]:

# def embed(docs: list[str],tokenizer= None,model= None,device = None) -> list[list[float]]:
#     doc_embeds = co.embed(
#         docs,
#         input_type="search_document",
#         model="embed-english-v3.0"
#     )
#     return doc_embeds.embeddings

### openai embedding function

In [12]:

# def embed(docs: list[str],tokenizer= None,model= None,device = None) -> list[list[float]]:
#     res = openai.embeddings.create(
#         input=docs,
#         model="text-embedding-ada-002"
#     )
#     doc_embeds = [r.embedding for r in res.data]
#     return doc_embeds




# # def get_embedding(text):
# #     return openai.Embedding.create(
# #         input=[text.replace('\n', ' ')],
# #         model='text-embedding-ada-002')['data'][0]['embedding']

### Building a Vector Index

Use this to build a Numpy array of cohere embedding vectors.

In [13]:
from tqdm.auto import tqdm
import numpy as np
import time


def VectorIndex_in_memory(chunks,tokenizer= None,model= None,device = None):
    LEN = len(chunks)
    batch_size = 256
    for i in tqdm(range(0, LEN, batch_size)):
        # start = time.time()   
        chunk_batch = chunks[i:min(i+batch_size,LEN)].copy()
        # embed current batch
        embed_batch = embed(chunk_batch,tokenizer=tokenizer,model=model,device=device)
        if i > 0: # add to existing np array if exists (otherwise create)
            # embed_batch is the new batch of embeddings (same size as chunk_batch)
            # arr = np.concatenate([arr, embed_batch.copy()])
            arr += embed_batch.copy()
        if i == 0:
            arr = embed_batch.copy()
        # print(f'batch time: {time.time() - start}')
        if i>10:
            break
    print(i,len(arr) )   
    return arr
# arr = VectorIndex_in_memory(chunks = data["chunk"],tokenizer=tokenizer,model=model,device=device)




In [14]:
from add_data_to_db import VectorIndexUpdate
text, embedding = VectorIndexUpdate(texts= data["chunk"],tokenizer=tokenizer,model=model,device=device)

  0%|          | 1/20792 [00:00<2:26:59,  2.36it/s]

Error while writing to DB relation "embeddings" does not exist
LINE 1: INSERT INTO embeddings (embedding, text) VALUES (ARRAY[ -0.3...
                    ^



  0%|          | 2/20792 [00:00<2:28:03,  2.34it/s]

Error while writing to DB relation "embeddings" does not exist
LINE 1: INSERT INTO embeddings (embedding, text) VALUES (ARRAY[ -0.1...
                    ^



  0%|          | 3/20792 [00:01<2:29:10,  2.32it/s]

...

Error while writing to DB relation "embeddings" does not exist
LINE 1: INSERT INTO embeddings (embedding, text) VALUES (ARRAY[0.002...
                    ^



  0%|          | 12/20792 [00:04<2:17:45,  2.51it/s]

Error while writing to DB relation "embeddings" does not exist
LINE 1: INSERT INTO embeddings (embedding, text) VALUES (ARRAY[0.036...
                    ^



  0%|          | 12/20792 [00:05<2:30:48,  2.30it/s]


KeyboardInterrupt: 

In [ ]:
# text.replace("\x00", "\uFFFD")
# text.decode("utf-8", errors="replace").replace("\x00", "\uFFFD")
text

Now we need to create the query mechanism, this is simply a cosine similarity calculation between a query vector and our arr vectors.

In [ ]:
# from numpy.linalg import norm

# # convert chunks list to array for easy indexing
# chunk_arr = np.array(chunks)

# def query_in_memory(text: str, top_k: int=3) -> list[str]:
#     # create query embedding
#     xq = embed([f"query: {text}"])[0]
#     # calculate cosine similarities
#     sim = np.dot(arr, xq.T) / (norm(arr, axis=1)*norm(xq.T))
#     # get indices of top_k records
#     idx = np.argpartition(sim, -top_k)[-top_k:]
#     docs = chunk_arr[idx]
#     for d in docs.tolist():
#         print(d)
#         print("----------")
# # query_in_memory("why should I use llama 2?")

In [ ]:
from vector_search import query
texts = ["why should I use llama 2?"]
query(texts, model, tokenizer, device)



## Usecases

https://medium.com/@vladris/embeddings-and-vector-databases-732f9927b377

https://medium.com/@vladris/n-shot-learning-f9bc0d670a41


Q&A solution

In [ ]:
import json

embeddings = json.load(open('embeddings.json', 'r'))


def nearest_embedding(embedding):
    nearest, nearest_distance = None, 1

    for path, embedding2 in embeddings.items():
        distance = cosine_distance(embedding, embedding2)
        if distance < nearest_distance:
            nearest, nearest_distance = path, distance

    return nearest


chat = ChatTemplate(
    {'messages': [{'role': 'system', 'content': 'You are a Q&A AI.'},
                  {'role': 'system', 'content': 'Here are some facts that can help you answer the following question: {{data}}'},
                  {'role': 'user', 'content': '{{prompt}}'}]})


while True:
    prompt = input('user: ')
    if prompt == 'exit':
        break

    context = nearest_embedding(get_embedding(prompt))
    data = open(context, 'r').read()

    message = chat.completion(
        {'data': data, 'prompt': prompt}).choices[0].message

    print(f'{message.role}: {message.content}')